In [1]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("PopularHero")
sc = SparkContext(conf=conf)

In [16]:
from operator import add

In [5]:
def countCoOccurences(line):
    elements = line.split()
    # subtract 1 to remove the superhero of interest
    return (int(elements[0]), len(elements) - 1)

In [6]:
def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode("utf8"))

In [11]:
names = sc.textFile('../SparkData/MarvelNames.txt')
# instead of using a broadcast, you can use a key-value pair
# key is ID, value is the superhero name
namesRdd = names.map(parseNames)

In [12]:
namesRdd.top(5)

[(19428, b'AA2 38'),
 (19427, b'AA2 20'),
 (19426, b'AA2 30'),
 (19425, b'WI 25/2'),
 (19424, b'BIZADV 33')]

In [14]:
lines = sc.textFile('../SparkData/MarvelGraph.txt')
pairings = lines.map(countCoOccurences)

In [15]:
pairings.top(5)

[(6486, 3), (6485, 3), (6484, 3), (6483, 3), (6482, 3)]

In [17]:
totalFriendsByCharacter = pairings.reduceByKey(add)

In [19]:
flipped = totalFriendsByCharacter.map(lambda x: (x[1], x[0]))

In [21]:
# finds the max item in the RDD
mostPopular = flipped.max()

In [22]:
mostPopular

(1933, 859)

In [25]:
# use lookup() to access the names/id key-value RDD
mostPopularName = namesRdd.lookup(mostPopular[1])[0]

In [26]:
mostPopularName

b'CAPTAIN AMERICA'

Turns out Captain America is the most popular superhero in the Marvel Universe. He has the most co-occurences amongst all entrants.